# Merkle trees

[Merkle trees](https://en.wikipedia.org/wiki/Merkle_tree) enable efficient and cryptographically secure membership tests: a prover can convince a verifier that a given element `x` is part of a given set.

In [ ]:
# We're using https://github.com/Tierion/pymerkletools
import sys
!{sys.executable} -m pip install merkletools


In [2]:
from merkletools import MerkleTools

mt = MerkleTools(hash_type="sha256")
mt.add_leaf(["values", "stored", "in", "a", "merkle", "tree"], True)
mt.make_tree()

What is actually stored in the tree are the hashes of the leaves:

In [4]:
leaf = mt.get_leaf(1)
leaf

'87b04e58961f9a99d853d4046a0b5b793e7c3e4bbd21f5aca8fb17c20cdb1d8b'

Let's verify that the item at index 1 (the second leaf) indeed hashes to this value:

In [3]:
import hashlib

leaf_value="stored"

hashlib.sha256(leaf_value.encode('utf-8')).hexdigest()

'87b04e58961f9a99d853d4046a0b5b793e7c3e4bbd21f5aca8fb17c20cdb1d8b'

The "root hash" of the tree represents a compact cryptographic "commitment" to all the values stored in the tree. Anyone can use this root hash to later check if a given value was or was not part of the merkle tree:

In [5]:
merkle_root = mt.get_merkle_root()
merkle_root

'0ecb1a0ac5a67dddae57135da2ea20c4c8af7039137aa5d8db075c8d099db1d9'

A merkle proof represents the proof that a specific item is part of a given tree (with a given root hash). The proof is nothing more than a list of hashes, namely of all the intermediate nodes that sit between the leaf and the root:

In [6]:
merkle_proof = mt.get_proof(1)
merkle_proof

[{'left': '89445ea08b55421faa49919a5fd272e9a520f701b479d6084847e161ca5b7711'},
 {'right': '50eeb1643a181dce534c921e839592b443946cb85b657a11efade52485d999ff'},
 {'right': '487e8e3fb58ea5fc6855763fe7a918bda75f564dd0649d8c6b7aefb6f23bd094'}]

To verify (or validate) a merkle proof, one provides the proof, the data item one wants to check is part of the tree, and the root hash of the tree:

In [7]:
mt.validate_proof(merkle_proof, leaf, merkle_root)

True

In [8]:
invalid = "invalid item"
h = hashlib.sha256(invalid.encode('utf-8')).hexdigest()
mt.validate_proof(merkle_proof, h, merkle_root)

False